# n = 5: Autoregressive covariance structure 

We try to parameterize covariance Gamma only using two parameters rho and sigma2 as in the AR(1) structure.

For n = 5 this autoregressive model converges in 15 iterations using Quasi-Newton, 13 with Newton. 

Here we just initialize beta under independent GLM assumptions, sigma2 using MM-Algorithm letting rho = 0, then MOM using the empirical covariance between Y_1 and Y_2.

We set IPOPT convergence tolerance to 10^-6, with the adaptive mean option turned on. 

$$\mu_i = 5, \rho = 0.9, \sigma^2 = 0.1, n_i = 5$$

$$\Gamma_i = \sigma^2 * \begin{pmatrix} 1 & \rho & \rho^2 & \rho^3 & \rho^4\\ \rho & 1 & \rho & \rho^2 & \rho^3 \\ \rho^2 & \rho & 1 & \rho & \rho^2 \\ \rho^3 & \rho^2 & \rho & 1 & \rho \\ \rho^4 & \rho^3 & \rho^2 & \rho & 1\end{pmatrix} \forall i \in [1, N = 10,000]$$ 

In [1]:
using DataFrames, Random, GLM, GLMCopula, Test
using LinearAlgebra, BenchmarkTools, Revise

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 5
ρ = 0.9
σ2 = 0.1

V = zeros(n, n) # will store the AR(1) structure without sigma2

mean = 5

dist = Poisson

V = get_AR_cov(n, ρ, σ2, V)

# true Gamma
Γ = σ2 * V

5×5 Array{Float64,2}:
 0.1      0.09    0.081  0.0729  0.06561
 0.09     0.1     0.09   0.081   0.0729
 0.081    0.09    0.1    0.09    0.081
 0.0729   0.081   0.09   0.1     0.09
 0.06561  0.0729  0.081  0.09    0.1

In [2]:
vecd = [dist(mean) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Array{Array{Float64,1},1}:
 [3.0, 8.0, 7.0, 4.0, 2.0]
 [2.0, 6.0, 6.0, 6.0, 7.0]
 [3.0, 5.0, 5.0, 10.0, 3.0]
 [5.0, 6.0, 3.0, 5.0, 2.0]
 [1.0, 10.0, 1.0, 2.0, 2.0]
 [5.0, 5.0, 4.0, 5.0, 4.0]
 [1.0, 4.0, 6.0, 5.0, 5.0]
 [7.0, 8.0, 6.0, 4.0, 9.0]
 [5.0, 3.0, 4.0, 6.0, 4.0]
 [5.0, 5.0, 6.0, 4.0, 7.0]
 [3.0, 7.0, 6.0, 3.0, 6.0]
 [6.0, 3.0, 3.0, 8.0, 5.0]
 [3.0, 4.0, 7.0, 3.0, 4.0]
 ⋮
 [7.0, 4.0, 8.0, 5.0, 3.0]
 [2.0, 4.0, 4.0, 7.0, 7.0]
 [7.0, 2.0, 4.0, 6.0, 8.0]
 [7.0, 4.0, 8.0, 2.0, 6.0]
 [3.0, 6.0, 5.0, 5.0, 1.0]
 [3.0, 4.0, 4.0, 7.0, 7.0]
 [3.0, 4.0, 8.0, 2.0, 3.0]
 [2.0, 9.0, 7.0, 6.0, 6.0]
 [6.0, 4.0, 3.0, 4.0, 7.0]
 [6.0, 4.0, 4.0, 9.0, 6.0]
 [2.0, 3.0, 2.0, 5.0, 4.0]
 [4.0, 4.0, 2.0, 4.0, 4.0]

In [3]:
d = Poisson()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaARObs{T, D, Link}}(undef, N)

for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    gcs[i] = GLMCopulaARObs(y, X, d, link)
end

gcm = GLMCopulaARModel(gcs);

In [4]:
initialize_model!(gcm)
@show gcm.β
@show exp.(gcm.β)
@show gcm.ρ
@show gcm.σ2;

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -112233.55795434186 39999
2 -112233.55795434186 -112233.55795434186 39999
gcm.β = [1.620377852472435]
exp.(gcm.β) = [5.0550000000000015]
gcm.ρ = [1.0]
gcm.σ2 = [0.08634406601451637]


In [5]:
Y_1 = [Y_Nsample[i][1] for i in 1:N]
Y_2 = [Y_Nsample[i][2] for i in 1:N]

update_rho!(gcm, Y_1, Y_2)
@show exp.(gcm.β)
@show gcm.ρ
@show gcm.σ2;

exp.(gcm.β) = [5.0550000000000015]
gcm.ρ = [0.9330886394490602]
gcm.σ2 = [0.08634406601451637]


In [6]:
loglikelihood!(gcm, true, true)

gcm1 = deepcopy(gcm)
gcm2 = deepcopy(gcm);

## Quasi Newton
## Number of Iterations....: 12
## 0.92 seconds 

In [7]:
@time GLMCopula.fit!(gcm1, IpoptSolver(print_level = 5, max_iter = 100, tol = 10^-6, mu_strategy = "adaptive", mu_oracle = "loqo", hessian_approximation = "limited-memory"))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equal

-112013.00953147069

# Newton
## Number of Iterations....: 14
## 0.84 seconds 

In [8]:
@time GLMCopula.fit!(gcm2, IpoptSolver(print_level = 5, max_iter = 100, tol = 10^-6, mu_strategy = "adaptive", mu_oracle = "loqo",  hessian_approximation = "exact"))

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        3
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

-112013.00953147092

In [9]:
@show gcm1.∇θ
@show gcm2.∇θ;

gcm1.∇θ = [1.5341795744916453e-6, 3.711056965616244e-7, 7.182926786697408e-7]
gcm2.∇θ = [2.8932820441696094e-6, -8.06504879546921e-9, -2.157918120637703e-6]


In [10]:
@show gcm1.θ
@show gcm2.θ;

gcm1.θ = [1.6123212633809156, 0.8695969631187371, 0.09778052310458214]
gcm2.θ = [1.612321263364836, 0.8695969629750789, 0.0977805232498743]


In [11]:
@show loglikelihood!(gcm1, true, true)
@show loglikelihood!(gcm2, true, true);

loglikelihood!(gcm1, true, true) = -112013.00953147069
loglikelihood!(gcm2, true, true) = -112013.00953147092
